# Car efficiency

# Problem 1

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

def xlsx_to_csv_pd(path):
    data_train = pd.read_excel(path + '15.xlsx')
    xls_16 = pd.read_excel(path + '16.xlsx')
    xls_17 = pd.read_excel(path + '17.xlsx')
    data_train = pd.concat([data_train,xls_16,xls_17])
    data_test = pd.read_excel(path + '18.xlsx')
    return data_train, data_test

def x_y_split(df):
    y = df.pop('Comb Unrd Adj FE - Conventional Fuel')
    return df,y

def choose_column(df):
    count = df.shape[0] - df.count()
    filtered =  count[count < len(df)*1]
    index = list(filtered.index)
    index = [col for col in index if col.find('EPA') == -1 and col.find('FE') == -1 and col.find('MPG') == -1 and col.find('CO2') == -1 and col.find('Smog') == -1 and col.find('Guzzler') == -1 and col.find('Release Date') == -1 and col.find('Mfr Name') == -1 and col.find('Verify Mfr Cd') == -1]
    return df[index], index

#load data
train, test = xlsx_to_csv_pd('/Users/haikundu/Desktop/4995AML/hw3/')

#data target split
train_x, train_y = x_y_split(train)
test_x, test_y = x_y_split(test)

#choose column
filtered_train, column_name = choose_column(train_x)
filtered_test = test[column_name]

### Preprocessing
After loading data, in the data cleaning part, first we fill up missing value by mean for numerical variable and mode for categorical variable. After that we use one hot encoding method to transform all categorical data into numerical data. Finally, we applied standard scaling method on all traing and test set.

In [2]:
def fillna_mean(dfo,index,dfto):
    df = pd.DataFrame()
    dft = pd.DataFrame()
    for col in index:
        if dfo[col][0].dtype == 'int64' or dfo[col][0].dtype == 'float64':
            df[col] = dfo[col].fillna(dfo.mean()[col])
            dft[col] = dfto[col].fillna(dfo.mean()[col])
        else:
            df[col] = dfo[col].fillna(dfo.mode()[col].tolist()[0]).tolist()
            dft[col] = dfto[col].fillna(dfo.mode()[col].tolist()[0]).tolist()
    return df,dft

def one_hot_encoding(train,test):
    full = pd.concat([train, test]) 
    extended = pd.get_dummies(full)
    boundry = len(train)
    train = extended[:boundry]
    test = extended[boundry:]
    return train, test

#fill up missing value
full_train,full_test = fillna_mean(filtered_train,column_name,filtered_test)

#transform categorical data into numerical data
ohe_train,ohe_test = one_hot_encoding(full_train, full_test)

#Standard Scaler
scaler = StandardScaler()
scaler.fit(ohe_train)
X_train_scaled = scaler.transform(ohe_train)
X_test_scaled = scaler.transform(ohe_test)

### Fitting model
Since this problem is a regression problem, in this section we will try three linear regression algorithm to predict the Conventional Fuel.

In [3]:
#fit ridge regression
print("Ridge validation score:",np.mean(cross_val_score(Ridge(), X_train_scaled, train_y, cv= 10)))

Ridge validation score: 0.9597902238073752


In [4]:
#fit lasso regression
print("Lasso validation score:",np.mean(cross_val_score(Lasso(), X_train_scaled, train_y, cv= 10)))

Lasso validation score: 0.6880833115008871


In [5]:
#fit Elastic Net regression
print("ElasticNet validation score:",np.mean(cross_val_score(ElasticNet(), X_train_scaled, train_y, cv= 10)))

ElasticNet validation score: 0.7909909848607104


Based on the validation score for those three models, we can find that Ridge regerssion performs best in three models.
Nest, we will use grid search for the best parameter and exam it performance on test set.

In [6]:
# grid search for best parameter and exam the performance of ridge regression model on test set
param_grid_ridge = {'alpha': np.logspace(-3, 3, 20)}
grid_ridge = GridSearchCV(Ridge(),param_grid=param_grid_ridge,cv=10,
                          return_train_score=True)
grid_ridge.fit(X_train_scaled, train_y)
print("best mean cross-validation score: {:.3f}".format(grid_ridge.best_score_))
print("best parameters: {}".format(grid_ridge.best_params_))
print("test-set score: {:.3f}".format(grid_ridge.score(X_test_scaled, test_y)))

best mean cross-validation score: 0.966
best parameters: {'alpha': 12.742749857031322}
test-set score: 0.954
